In [12]:
import asdf
import numpy as np

# 08 More About ASDF

**Outline**
- Objects intrinsically handled by ASDF
- Building the ASDF tree
- Serializing everything else
- A tour of the asdf-format organization

## Objects Handled Intrinsically by ASDF

Recall that ASDF files store their information using a tree (nested key/value) structure.
This allows the stored information to be hierarchically organized with the file. Fundamentally,
this tree is a nested combination of basic structures:
- maps (forming the tree structure itself),
- lists,
- arrays,
- strings,
- Booleans,
- and numbers.

All of which are stored using `yaml`. Note that more complex structures are denoted using
`yaml` tags, but those tagged structures are still comprised of the above basic structures,
and that these additional tagged objects are generally supported via ASDF extensions.

The Python analogs to these basic structures are:
- maps -> `dict`
- lists -> `list`
- arrays -> `np.ndarray`
- strings -> `str`
- Booleans -> `bool`
- numbers -> `int`, `float`, `complex` (depending on type of number).

One special note is that although `yaml` supports a more general notion of what keys
are allowable within a map, the limitations of Python have restricted ASDF to only allowing
`bool`, `int`, or `str` types as keys.

## Building the Tree

To create an ASDF, one must build a tree comprised of key/value pairs. Typically,
this takes the form of creating a Python `dict` which is then passed to ASDF to
be written to a file.

Let's start by creating a simple tree consisting of a `"hello": "world"` key/value
pair and then writing it to an ASDF file.

In [13]:
tree = {"hello": "world"}
af = asdf.AsdfFile(tree)
af.write_to("hello.asdf")

Performing a cursory inspection of the `hello.asdf` file indicates that it indeed contains
our simple tree.

In [14]:
with open("hello.asdf") as f:
    print(f.read())

with asdf.open("hello.asdf") as af:
    assert af["hello"] == tree["hello"]

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
hello: world
...



### Exercise 1

Create a tree with an entry for all the basic Python types supported by ASDF except `np.ndarray`
(these are handled in a unique fashion).

### Serializing `np.ndarray`

Recall that in addition to the `yaml` data in an ASDF file, there can also be
a series of binary blocks storing data in the file. These blocks of data typically
correspond to storing array structures in the ASDF file.

In Python, these array structures are usually `np.ndarray` objects as these objects
are extremely flexible in how they can store complex array-like data. Just as with
the other objects described above, serializing `np.ndarray` is seamless; meaning that,
ASDF will serialize these objects within the `tree` passed to ASDF without any additional
work. However, normally the actual data contained within the `np.ndarray` will be stored
within a binary block instead of within the `yaml` metadata (there is an option to "inline"
this data so that it is stored within the `yaml`, but this is not the default).

For example serializing a `np.ndarray` looks like:

In [15]:
tree = {"array": np.random.rand(8, 8)}
af = asdf.AsdfFile(tree)
af.write_to("array.asdf")

Performing another cursory inspection of the `array.asdf` file:

In [16]:
with open("array.asdf", "r", encoding="unicode_escape") as f:
    print(f.read())

with asdf.open("array.asdf") as af:
    assert np.all(af["array"] == tree["array"])

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
array: !core/ndarray-1.0.0
  source: 0
  datatype: float64
  byteorder: little
  shape: [8, 8]
...
ÓBLK 0                             Ô/îd¯[Ë]»&Úó'ï?>ÄvÅ2Ý?¨|;x(?½?&»±ÇÐí?@Ýÿ4Ú?ãyj½ê?¤?3åÖ?®,àAá?ntaA{Uì?ÔXY¯øÄ?oíA	N²ä?uNd¹Äà?4]<ÃÈÇ?Ì­­'!!ê?T:Ò-§Ê?«¿ç?ÞghÔÞ?´²rþ´Þ?ºFÊÆOë?# ´ä?àe+c¬Pâ?T¬¬=QÅ?Ö\ïç°ä?z±ÂÄ±1î?@g Ù>'Ô? if"Ú?gÀ¿lÆ?àÇLÔ_´dÃÚ?¼9ÁOuãÍ?ô§xË?vVllÐÂÜ?ÚMgaª?)Ã[É?îìÑÚÕÕ?\ÌÛ9·É?gú¢ï?p²`5Ú§?èA®³?°»jè?7¼|[°ä?LùVß|å]oÓéÄ?Ç4aÅÛ¢à?6u!ä¹×?àtß?@/ùªÖå?¶f³³å?^¡Éä?ÐÔ¿ÖÒ½? üb&º?§äÛÝ«Òá?¹Æ¦<Iãâ?T¶¡hê? BÈ¥Ò?

ASDF will store this data in an efficient manner. By this we mean that arrays shared between
different objects stored in the ASDF tree, will only be stored once as a binary block with
both entries in the `yaml` metadata will both reference the same binary block. Moreover,
this extends to objects which reference a different view of the same data, meaning the views
will all still reference the same binary block, only storing information on the view itself.

### Exercise 2

Create tree containing the same `np.ndarray` twice, and multiple views on the same `np.ndarray`.

## Serializing Everything Else

As mentioned above, other types of objects can also be serialized by ASDF, but these
objects are denoted in the `yaml` metadata via a `yaml` tag. Indeed some of the objects
already discussed are tagged in the metadata. Typically, these more complex objects
are supported via ASDF extensions (the creation of which will be discussed in a later
notebook), and when an extension which supports a particular object is installed, ASDF
will seamlessly serialize that object when it is placed within the ASDF tree.

In practice, this "seamless" interaction with the object is handled by the ASDF extension,
which will essentially specify a "sub-tree" (which is tagged by a `yaml` tag) which represents
that object, wherein the extension knows how to turn the object into that "sub-tree" and read
that "sub-tree" in order to reconstruct the object.

### Example - Serializing `astropy` Objects

The Roman pipeline makes extensive use of the `astropy` library, which includes several
objects that are useful to astronomical data. Due to this, the `asdf-astropy` extension
library (discussed later) provides extensions for most `astropy` objects to be serialized
and/or deserialized from ASDF files. Note that `asdf-astropy` is a dependency of the
Roman pipeline, so it will always be available for Roman ASDF work.

Currently `asdf-astropy` supports the following general object categories:

- `astropy` units and `quantities`.
- Most `astropy` models.
- `astropy` time objects.
- `astropy` coordinate objects.

For example, lets serialize `astropy`'s `Gaussian1D` model:

In [17]:
import astropy.modeling.models as models

tree = {"gaussian": models.Gaussian1D(1, 2)}
af = asdf.AsdfFile(tree)
af.write_to("gaussian.asdf")

Performing another cursory inspection of the `gaussian.asdf` file:

In [18]:
with open("gaussian.asdf") as f:
    print(f.read())

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension._manifest.ManifestExtension
    extension_uri: asdf://asdf-format.org/transform/extensions/transform-1.5.0
    software: !core/software-1.0.0 {name: asdf-astropy, version: 0.2.1}
gaussian: !transform/gaussian1d-1.0.0
  amplitude: 1.0
  bounding_box: [-3.5, 7.5]
  bounds:
    stddev: [1.1754943508222875e-38, null]
  inputs: [x]
  mean: 2.0
  outputs: [y]
  stddev: 1.0
...



#### Exercise 3

Pick a few `astropy` objects and create an ASDF file containing them.

## A Tour of the asdf-format organization

The asdf-format organization on GitHub is where the majority of the ASDF related
related repositories/packages are located. These include:

- [`asdf`](https://github.com/asdf-format/asdf)
  - The actual library.
- [`asdf-standard`](https://github.com/asdf-format/asdf-standard)
  - The base schemas and description of the functionality of asdf.
- [`asdf-transform-schemas`](https://github.com/asdf-format/asdf-transform-schemas)
  - The schemas for mathematical functions/models (typically for `astropy` models).
- [`asdf-coordinates-schemas`](https://github.com/asdf-format/asdf-coordinates-schemas)
  - The schemas for coordinates (typically `astropy` coordinates).
- [`asdf-wcs-schemas`](https://github.com/asdf-format/asdf-wcs-schemas)
  - The schemas for WCS objects (typically `gwcs` WCS).
- [`asdf-astropy`](https://github.com/astropy/asdf-astropy)
  - Not technically part of asdf-format, but maintained by ASDF maintainers as a
  collaboration with the astropy organization.
  - Contains schemas for some specialized `astropy` objects (not in the other packages).
  - Contains extensions for support of `astropy` objects.